# Scikit-learn K折交叉驗證法

## 載入相關套件

In [1]:
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np

## 載入資料集

In [2]:
X, y = datasets.load_diabetes(return_X_y=True)

## 資料分割

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

## 特徵縮放

In [4]:
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

## 模型訓練

In [5]:
from sklearn.linear_model import LinearRegression

clf = LinearRegression()
clf.fit(X_train_std, y_train)

LinearRegression()

## 模型評分

In [6]:
print(f'R2={clf.score(X_test_std, y_test)}')

R2=0.4426466357761497


## K折測試

In [7]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5)
for i, (train_index, test_index) in enumerate(kf.split(X_train_std)):
    print(f"Fold {i}:") 
    print(f"  Train: index={train_index}")
    print(f"  Test:  index={test_index}")

Fold 0:
  Train: index=[ 71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88
  89  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106
 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124
 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142
 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160
 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178
 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196
 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214
 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232
 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250
 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268
 269 270 271 272 273 274 275 276 277 278 279 280 281 282 283 284 285 286
 287 288 289 290 291 292 293 294 295 296 297 298 299 300 301 302 303 304
 305 306 307 308 309 310 311

## K折驗證

In [8]:
score = []
for i, (train_index, test_index) in enumerate(kf.split(X_train_std)):
    X_new = X_train_std[train_index]
    y_new = y_train[train_index]
    clf.fit(X_new, y_new)
    score_fold = clf.score(X_train_std[test_index], y_train[test_index])
    score.append(score_fold)
    print(f"Fold {i} 分數: {np.mean(score)}") 
print(f"平均值: {np.mean(score)}")
print(f"標準差: {np.std(score)}")

Fold 0 分數: 0.6666917365897675
Fold 1 分數: 0.550452952018643
Fold 2 分數: 0.46037623756274487
Fold 3 分數: 0.48249540993541384
Fold 4 分數: 0.47270352560851314
平均值: 0.47270352560851314
標準差: 0.12922423179188272


## 效能調校

In [9]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

lasso = Lasso(random_state=0, max_iter=10000)

# 正則化強度：30種選擇
alphas = np.logspace(-4, -0.5, 30)
# 強迫係數(權重)須為正數
positive = (True, False)
tuned_parameters = [{"alpha": alphas, 'positive':positive}]

# 效能調校
clf = GridSearchCV(lasso, tuned_parameters, cv=5, refit=False)
clf.fit(X, y)

LL= LinearRegression()
tuned_parameters2 = [{'positive':positive}]
clf2 = GridSearchCV(LL,tuned_parameters2, cv=5)
clf2.fit(X, y)

scores_mean = clf.cv_results_["mean_test_score"]
scores_std = clf.cv_results_["std_test_score"]
print('平均分數:\n', scores_mean, '\n標準差:\n', scores_std)

scores_mean2 = clf2.cv_results_["mean_test_score"]
scores_std2 = clf2.cv_results_["std_test_score"]
print('平均分數2:\n', scores_mean2, '\n標準差2:\n', scores_std2)

平均分數:
 [0.45536804 0.48231819 0.45536837 0.48231859 0.45536901 0.48231898
 0.4553696  0.48231929 0.45537035 0.48231932 0.45537113 0.4823187
 0.45537247 0.48231675 0.45537418 0.48231218 0.45537643 0.48230267
 0.45537939 0.48236223 0.45538327 0.48245888 0.45538829 0.48247867
 0.45539479 0.48250645 0.45540315 0.4825296  0.45541379 0.48245452
 0.45542718 0.48201034 0.45544378 0.48130789 0.45546905 0.48115831
 0.45551819 0.4814086  0.45557755 0.48175196 0.45564629 0.48188457
 0.45572024 0.48209859 0.45578861 0.4820597  0.45582787 0.4819337
 0.45579065 0.48108101 0.4555864  0.47915084 0.45504625 0.47635391
 0.45386216 0.47235481 0.45147624 0.46555482 0.44692952 0.45663349] 
標準差:
 [0.05546675 0.04924557 0.05546542 0.04923826 0.05546391 0.04922864
 0.05546161 0.04921601 0.05545855 0.04919946 0.05545488 0.04917782
 0.05544958 0.04914961 0.05544256 0.04911303 0.05543329 0.04906587
 0.05542106 0.04898322 0.05540492 0.04888308 0.0553836  0.04877681
 0.05535546 0.04866801 0.05531833 0.04856384 0.05

In [10]:
# 取得最高分數
np.max(clf.cv_results_["mean_test_score"])

0.4825295987936247

In [11]:
# 參數組合
clf.param_grid

[{'alpha': array([1.00000000e-04, 1.32035178e-04, 1.74332882e-04, 2.30180731e-04,
         3.03919538e-04, 4.01280703e-04, 5.29831691e-04, 6.99564216e-04,
         9.23670857e-04, 1.21957046e-03, 1.61026203e-03, 2.12611233e-03,
         2.80721620e-03, 3.70651291e-03, 4.89390092e-03, 6.46167079e-03,
         8.53167852e-03, 1.12648169e-02, 1.48735211e-02, 1.96382800e-02,
         2.59294380e-02, 3.42359796e-02, 4.52035366e-02, 5.96845700e-02,
         7.88046282e-02, 1.04049831e-01, 1.37382380e-01, 1.81393069e-01,
         2.39502662e-01, 3.16227766e-01]),
  'positive': (True, False)}]

In [12]:
# 取得最佳參數組合
clf.best_params_

{'alpha': 0.0037065129109221566, 'positive': False}

In [15]:
# 驗證
from math import floor
index = np.argmax(clf.cv_results_["mean_test_score"])
index, clf.cv_results_["mean_test_score"][index], alphas[floor((index-1)/2)]

(27, 0.4825295987936247, 0.0037065129109221566)

In [13]:
# 最佳分數
clf.best_score_

0.4825295987936247

## 以最佳參數組合重新訓練

In [17]:
clf = Lasso(random_state=0, max_iter=10000, alpha=0.0037065129109221566
            , positive=False)
clf.fit(X_train_std, y_train)
clf.score(X_test_std, y_test)

0.507827542194291